# Function Calling

In [ ]:
import importlib

if not importlib.util.find_spec("class_utils"):
    !pip install -qqq git+https://github.com/xtreamsrl/genai-for-engineers-class

Function calling for language models (LLMs) allows developers to define and invoke specific functions within the model’s output. 

In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. 

The LLM **does not call** the function; instead, the model generates JSON that you can use to call the function in your code.

This feature enhances the precision and control over the responses generated by the LLM, enabling more accurate and relevant outputs. By specifying a function, developers can ensure the LLM processes tasks such as data extraction, content generation, or API interactions in a structured and predictable manner. 

This is especially useful in applications where consistency and correctness are critical, such as automated customer support, dynamic content generation, and complex data manipulation. 

Overall, function calling transforms LLMs from simple text generators into versatile tools that can execute predefined tasks, improving reliability and efficiency in various development scenarios.

# Setup: packages and environment variables

In [ ]:
import os
import json
from pprint import pprint

import httpx

from haystack import Document
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever

from class_utils.data import get_movie_dataset_as_documents
from class_utils.haystack_pipelines import (
    build_indexing_pipline,
    build_prompt_building_pipeline,
)
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = ...
os.environ["TOKENIZERS_PARALLELISM"] = "true"
THE_MOVIE_DB_BEARER = ...

# Indexing

Let's run our usual indexing pipeline without any modification.

However, this time we only load 5 documents, because we want the query to find no matches.

In [ ]:
documents = get_movie_dataset_as_documents(5)
document_store = QdrantDocumentStore(":memory:", embedding_dim=384)
indexing_pipeline = build_indexing_pipline(document_store)
indexing_pipeline.run({"doc_embedder": {"documents": documents}})

We also hard-code the template, to focus on the function calling and not on other topics.

In [ ]:
template = """
Answer the questions based on the given context.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
Question: {{ question }}
Answer:
"""

Next, we run the pipeline. Please pay attention to the query.

In [ ]:
prompt_building_pipe = build_prompt_building_pipeline(
    QdrantEmbeddingRetriever(document_store), template
)

query = "What film talks about the atomic bomb? If you find no matching movies in the context, use your tools and functions to search for some."
prompt_builder_output = prompt_building_pipe.run(
    {"embedder": {"text": query}, "prompt_builder": {"question": query}}
)
prompt = prompt_builder_output.get("prompt_builder").get("prompt")
pprint(prompt)

There are no movies specifically about the atomic bomb.

However, there are countless other movies out there. If only our language model could access an external service...

It turn out it can! OpenAI supports Function Calling: https://platform.openai.com/docs/guides/function-calling.

We can provide the model with some definitions of functions and let the model decide whether to call them.

We must note that the model **DOES NOT** execute the function itself, that is up to us. The model just answer with an indication of the function to execute and the parameters it would use.

Let's try and make GPT aware of our function `search_for_movies` below. We should not ask the model to choose the value of the bearer token, as we will use our own key.

# Configuring openai for function calling

In [ ]:
client = OpenAI()
messages = [
    {
        "role": "system",
        "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    },
    {
        "role": "user",
        "content": prompt,
    },
]
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_for_movies",
            "description": "Search for movies using external APIs based on a query string.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query to search for movies.",
                    },
                },
                "required": ["query"],
            },
        },
    }
]
response = client.chat.completions.create(
    messages=messages, model="gpt-4.1", parallel_tool_calls=False, tools=tools
)
pprint(response)

And here is the implementation of the function.

In [ ]:
def search_for_movies(query: str, bearer_token: str) -> list[Document]:
    response = httpx.get(
        url="https://api.themoviedb.org/3/search/movie",
        params={
            "include_adult": False,
            "language": "en-US",
            "query": query,
            "region": "US",
        },
        headers={
            "accept": "application/json",
            "Authorization": f"Bearer {bearer_token}",
        },
    )
    docs = [
        Document(
            id=movie["id"],
            content=f"title: {movie['original_title']} \noverview: {movie['overview']}",
            meta={
                "title": movie.get("original_title"),
                "release_date": movie.get("release_date"),
            },
        )
        for movie in response.json()["results"]
    ]
    return docs

# Performing the actual function call
Now we must interpret the directions of the model. 

If it wants to call our function, we will extract the arguments and the execute `search_for_movies` with them.

Then, we will properly format the return values in a new prompt and we will pass it to the model. 

This should allow GPT to answer our initial query.

We will not do it, but please consider validating and sanitising the arguments to be passed to your functions. The LLMs may product invalid inputs, that may crash your code.

In [ ]:
if tool_calls := response.choices[0].message.tool_calls:
    print("Tool call detected")
    call = tool_calls[0]
    if call.function.name == "search_for_movies":
        query = json.loads(call.function.arguments).get("query")
        print(f"Searching for movies with query: {query}")
        movies = search_for_movies(query, THE_MOVIE_DB_BEARER)
        print(f"Found {len(movies)} movies")
        messages.append(response.choices[0].message)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": call.id,
                "name": call.function.name,
                "content": "\n".join([m.content for m in movies]),
            }
        )
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4.1",
            messages=messages,
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Unknown function call: {call.function.name}")